In [13]:
print("""Data source: 
    lingyun-template.xlsx, 
    lingyun-output.xlsx""")

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

Data source: 
    lingyun-template.xlsx, 
    lingyun-output.xlsx


In [14]:
data_explode = pd.read_excel('lingyun-output.xlsx')
data_explode

反馈日期        客诉来源 用户评级                     投诉事项     区域  投诉事项-source  \
0   2023-06-05  sitejabber   C5                活动问题、客服问题     美国         活动问题   
1   2023-06-05  sitejabber   C5                活动问题、客服问题     美国         客服问题   
2   2023-06-05  sitejabber    无      优惠券使用问题、退货退款问题-退款问题  未识别区域      优惠券使用问题   
3   2023-06-05  sitejabber    无      优惠券使用问题、退货退款问题-退款问题  未识别区域  退货退款问题-退款问题   
4   2023-06-05  sitejabber   C3                物流问题-签收未收     美国    物流问题-签收未收   
..         ...         ...  ...                      ...    ...          ...   
525 2023-06-11         BBB   C0           广告问题-虚假宣传、客服问题     美国    广告问题-虚假宣传   
526 2023-06-11         BBB   C0           广告问题-虚假宣传、客服问题     美国         客服问题   
527 2023-06-11         BBB   C1                物流问题-签收未收     美国    物流问题-签收未收   
528 2023-06-11  Trustpilot    无  广告问题-价格虚假宣传、产品问题-访问权限问题     英国  广告问题-价格虚假宣传   
529 2023-06-11  Trustpilot    无  广告问题-价格虚假宣传、产品问题-访问权限问题     英国  产品问题-访问权限问题   

    投诉事项-split  
0           活动  
1           客服  
2          优惠券  
3         退货退款  
4           物流  
..         ...  
525         广告  
526         客服  
527         物流  
528         广告  
529         产品  

[530 rows x 7 columns]

In [15]:
rename_dict = {'投诉事项-split': '问题类型', '客诉来源': '问题个数'}
frame = data_explode.groupby(['反馈日期', '区域', '投诉事项-split']).count().iloc[:, 0].reset_index().rename(rename_dict, axis=1)
frame

反馈日期   区域  问题类型  问题个数
0   2023-06-05   德国    商品     2
1   2023-06-05   德国    客服     1
2   2023-06-05   德国    物流     3
3   2023-06-05   德国    评论     1
4   2023-06-05   德国  退货退款     1
..         ...  ...   ...   ...
268 2023-06-11   英国    广告     1
269 2023-06-11   英国    活动     1
270 2023-06-11   英国    物流     1
271 2023-06-11   英国  退货退款     1
272 2023-06-11  西班牙    订单     1

[273 rows x 4 columns]

In [16]:
frame_all = frame.groupby(['反馈日期', '问题类型']).sum()['问题个数'].reset_index().reindex(frame.columns, axis=1)
frame_all['区域'] = '所有区域'
frame = pd.concat([frame, frame_all], axis=0, ignore_index=True)

In [19]:
with pd.ExcelWriter('lingyun-area.xlsx', mode='w', engine='openpyxl') as writer:
    start = min(frame['反馈日期']).strftime(r'%Y-%m-%d')
    end = max(frame['反馈日期']).strftime(r'%Y-%m-%d')
    for area in frame['区域'].unique():
        frame_area = frame[frame['区域'] == area]

        # Get pivot table
        frame_pivot = frame_area.pivot(index='问题类型', columns='反馈日期', values='问题个数')
        frame_pivot = frame_pivot.reindex(pd.date_range(start, end, freq='1D'), axis=1)
        frame_pivot.columns = pd.date_range(start, end, freq='1D').map(lambda x: x.strftime(r'%Y-%m-%d'))
        frame_pivot = frame_pivot.sort_index(key=lambda x: frame_pivot.sum(axis=1)[x], ascending=False)

        # Save to excel sheet
        frame_pivot_print = frame_pivot.copy()
        frame_pivot_print['总计'] = frame_pivot_print.sum(axis=1)
        frame_pivot_print.loc['总计'] = frame_pivot_print.sum(axis=0)
        frame_pivot_print.to_excel(writer, area)

        # Melt the pivot table and plot
        frame_melt = frame_pivot.reset_index().melt(id_vars='问题类型')
        frame_melt.columns = '问题类型 反馈日期 问题个数'.split()
        fig = go.Figure()
        topn = 10
        names_plot = frame_pivot.index[:topn] if len(frame_pivot.index) >= topn else frame_pivot.index
        for name in names_plot:
            group = frame_melt[frame_melt['问题类型'] == name]
            fig.add_trace(go.Scatter(
                x=group['反馈日期'],
                y=group['问题个数'],
                name=name,
                line=dict(shape='spline')
            ))
        title = area + '问题类型每日趋势 ' + start + '~' + end
        fig.update_layout(
            title=title, 
            legend=dict(orientation='h', yanchor='bottom', y=-.25),
            height=500,
            width=1000
        )
        fig.write_image('area-pictures/' + area + '.png', scale=1.5)

In [17]:
writer = pd.ExcelWriter('lingyun-area.xlsx', mode='w', engine='openpyxl')
start = min(frame['反馈日期']).strftime(r'%Y-%m-%d')
end = max(frame['反馈日期']).strftime(r'%Y-%m-%d')
for area in frame['区域'].unique():
    frame_area = frame[frame['区域'] == area]

    # Get pivot table
    frame_pivot = frame_area.pivot(index='问题类型', columns='反馈日期', values='问题个数')
    frame_pivot = frame_pivot.reindex(pd.date_range(start, end, freq='1D'), axis=1)
    frame_pivot.columns = pd.date_range(start, end, freq='1D').map(lambda x: x.strftime(r'%Y-%m-%d'))
    frame_pivot = frame_pivot.sort_index(key=lambda x: frame_pivot.sum(axis=1)[x], ascending=False)

    # Save to excel sheet
    frame_pivot_print = frame_pivot.copy()
    frame_pivot_print['总计'] = frame_pivot_print.sum(axis=1)
    frame_pivot_print.loc['总计'] = frame_pivot_print.sum(axis=0)
    frame_pivot_print.to_excel(writer, area)

    # Melt the pivot table and plot
    frame_melt = frame_pivot.reset_index().melt(id_vars='问题类型')
    frame_melt.columns = '问题类型 反馈日期 问题个数'.split()
    fig = go.Figure()
    topn = 10
    names_plot = frame_pivot.index[:topn] if len(frame_pivot.index) >= topn else frame_pivot.index
    for name in names_plot:
        group = frame_melt[frame_melt['问题类型'] == name]
        fig.add_trace(go.Scatter(
            x=group['反馈日期'],
            y=group['问题个数'],
            name=name,
            line=dict(shape='spline')
        ))
    title = area + '问题类型每日趋势 ' + start + '~' + end
    fig.update_layout(
        title=title, 
        legend=dict(orientation='h', yanchor='bottom', y=-.25),
        height=500,
        width=1000
    )
    fig.write_image('area-pictures/' + area + '.png', scale=1.5)
writer._save()
writer.close()


## matplotlib 的尝试
- 中文不支持
- 无法做到 smoothing
- 放弃，还是使用 plotly 吧

In [18]:
# plt.rcParams['font.sans-serif'][0] = ['Songti SC', 'STKaiti', 'STSong', 'KaiTi'][0]

# start = min(frame['反馈日期']).strftime(r'%Y%m%d')
# end = max(frame['反馈日期']).strftime(r'%Y%m%d')

# for area in set(frame['区域']):
#     frame_area = frame[frame['区域'] == area]
#     plt.figure(figsize=(8, 6), dpi=200, facecolor='white')
#     for qt in frame_area.groupby('问题类型').sum().sort_values('问题个数', ascending=False).head(10).index:
#         frame_area_qt = frame_area[frame_area['问题类型'] == qt]
#         plt.plot(frame_area_qt['反馈日期'], frame_area_qt['问题个数'], '-o', label=qt)
#     plt.title(area + ' ' + start + '-' + end)
#     plt.legend()
#     plt.grid(ls=':')
#     # plt.savefig(area + ' ' + start + '-' + end + '.png')
    
#     frame_area.sort_values(['问题类型', '反馈日期']).to_excel(area + ' ' + start + '-' + end + '.xlsx')
#     break